<a href="https://colab.research.google.com/github/ayaan-anwar/fifa20-ratings/blob/master/fifa20_ratings_DNN_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [4]:
fifa = pd.read_csv('/content/drive/My Drive/FIFA_20.csv')
fifa.columns

Index(['overall', 'name', 'nationality', 'club', 'position', 'preferred_foot',
       'work_rate', 'jersey_number', 'wage_eur', 'international_reputation',
       'weak_foot', 'skill_moves', 'age', 'height_cm', 'weight_kg', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic', 'finishing',
       'fk_accuracy', 'acceleration', 'agility', 'stamina', 'penalties',
       'composure', 'standing_tackle', 'sliding_tackle'],
      dtype='object')

In [5]:
skill_cols = [col for col in fifa.columns if col not in ('name', 'nationality', 'club', 'position', 
                                                         'preferred_foot', 'work_rate', 'jersey_number', 
                                                         'international_reputation', 'weak_foot', 'skill_moves', 
                                                         'wage_eur', 'age', 'height_cm', 'weight_kg')]
fifa= pd.read_csv('/content/drive/My Drive/FIFA_20.csv', usecols=skill_cols)
fifa.head()

,overall,pace,shooting,passing,dribbling,defending,physic,finishing,fk_accuracy,acceleration,agility,stamina,penalties,composure,standing_tackle,sliding_tackle
0,94,87,92,92,96,39,66,95,94,91,93,75,75,96,37,26
1,93,90,93,82,89,35,78,94,76,89,87,85,85,95,32,24
2,92,91,85,87,95,32,58,87,87,94,96,81,90,94,26,29
3,91,91,83,86,94,35,66,84,79,94,95,84,88,91,27,22
4,91,76,86,92,86,61,78,82,83,77,78,89,79,91,58,51


# Train - Test Split

In [6]:
X = fifa.drop(['overall'], axis=1)
X

,pace,shooting,passing,dribbling,defending,physic,finishing,fk_accuracy,acceleration,agility,stamina,penalties,composure,standing_tackle,sliding_tackle
0,87,92,92,96,39,66,95,94,91,93,75,75,96,37,26
1,90,93,82,89,35,78,94,76,89,87,85,85,95,32,24
2,91,85,87,95,32,58,87,87,94,96,81,90,94,26,29
3,91,83,86,94,35,66,84,79,94,95,84,88,91,27,22
4,76,86,92,86,61,78,82,83,77,78,89,79,91,58,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16037,57,23,28,33,47,51,17,21,56,45,60,39,41,50,52
16038,58,24,33,35,48,48,20,29,55,43,55,37,35,53,57
16039,54,35,44,45,48,51,33,32,55,52,50,36,39,52,46
16040,59,35,47,47,45,52,34,39,55,55,42,36,40,44,54


In [7]:
Y = fifa['overall']
Y

0        94
1        93
2        92
3        91
4        91
         ..
16037    48
16038    48
16039    48
16040    48
16041    48
Name: overall, Length: 16042, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=42)

# Training the Model

In [9]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 166,913
Trainable params: 166,913
Non-trainable params: 0
_________________________________________________________________


In [10]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [11]:
NN_model.fit(x_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
277/281 [============================>.] - ETA: 0s - loss: 4.4735 - mean_absolute_error: 4.4735
Epoch 00001: val_loss improved from inf to 1.47880, saving model to Weights-001--1.47880.hdf5
281/281 [==============================] - 1s 4ms/step - loss: 4.4394 - mean_absolute_error: 4.4394 - val_loss: 1.4788 - val_mean_absolute_error: 1.4788
Epoch 2/500
264/281 [===========================>..] - ETA: 0s - loss: 1.6275 - mean_absolute_error: 1.6275
Epoch 00002: val_loss did not improve from 1.47880
281/281 [==============================] - 1s 3ms/step - loss: 1.6323 - mean_absolute_error: 1.6323 - val_loss: 2.5946 - val_mean_absolute_error: 2.5946
Epoch 3/500
266/281 [===========================>..] - ETA: 0s - loss: 1.7421 - mean_absolute_error: 1.7421
Epoch 00003: val_loss did not improve from 1.47880
281/281 [==============================] - 1s 3ms/step - loss: 1.7245 - mean_absolute_error: 1.7245 - val_loss: 1.6084 - val_mean_absolute_error: 1.6084
Epoch 4/500
272/281 [

In [17]:
# Load weights file of the best model :
weights_file = '/content/Weights-449--0.78073.hdf5' # choose the best checkpoint 
NN_model.load_weights(weights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

# Testing

In [18]:
y_predicted = NN_model.predict(x_test)
y_predicted

array([[59.853474],
       [71.248886],
       [70.075645],
       ...,
       [62.7249  ],
       [70.498665],
       [64.104576]], dtype=float32)

In [19]:
y_test

13449    60
4045     71
4147     71
15364    54
7624     67
         ..
11670    63
9997     64
11978    62
3094     72
9255     65
Name: overall, Length: 4813, dtype: int64

In [20]:
from sklearn.metrics import r2_score

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_predicted)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (mae / y_test)

# Calculate accuracy
accuracy = 100 - np.mean(mape)

print('Mean Absolute Error:', mae)
print('Accuracy:', round(accuracy, 2), '%.')
print('R2 Error: ', r2_score(y_test, y_predicted))

Mean Absolute Error: 0.8165539635111977
Accuracy: 98.76 %.
R2 Error:  0.9752406131576915
